## Helper script to convert data in .h5 to pytorch dataloader

In [1]:
from pyrl.utils.data import GDict
import torch
from torch.utils.data import Dataset, DataLoader
import tqdm as tqdm
import numpy as np
from mani_skill2.envs import *
import gym
import json

cupy is not installed.


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
h5_demo = GDict.from_hdf5("/home/yihe/ibc_torch/work_dirs/demos/hang_demo.none.pd_joint_delta_pos.h5")
json_data = json.load(open('/home/yihe/ibc_torch/work_dirs/demos/trajectory.none.pd_joint_delta_pos.json'))

In [8]:
env = gym.make("Hang-v0", obs_mode="particles", control_mode = "pd_joint_delta_pos")

Warp initialized:
   Version: 0.3.1
   CUDA device: NVIDIA GeForce GTX 1080 Ti
   Kernel cache: /home/yihe/.cache/warp/0.3.1


2022-08-05 16:43:59,575 - mani_skill2 - INFO - load sdfs from file


In [16]:
observations = []
actions = []

In [17]:
episode_count = 0

for episode in json_data['episodes']:
    episode_id = 'traj_' + str(episode['episode_id'])
    env.reset(seed=episode["episode_seed"])

    for action in h5_demo[episode_id]['actions']:
        obs = env.get_obs()
        xyz = obs['particles']['x'][np.random.choice(range(len(obs['particles']['x'])), size=1024, replace=False)]
        agent = np.hstack((obs['agent']['qpos'], obs['agent']['qvel'], env.rod.get_pose().p,env.rod.get_pose().q))
        observations.append((xyz, agent))
        actions.append(action)
    
    episode_count += 1
    if episode_count == 10:
        break

In [23]:
np.save('/home/yihe/ibc_torch/work_dirs/demos/hang_obs.npy', np.array(observations, dtype=object))
np.save('/home/yihe/ibc_torch/work_dirs/demos/hang_act.npy', actions)

In [4]:
h5_demo['traj_0']['actions'][0]
h5_demo['traj_0']['obs']['xyz'][0].shape

(2048, 3)

In [7]:
h5_demo['traj_0']['actions'].max()

1.0

In [5]:
env = gym.make("Hang-v0", obs_mode="particles", control_mode = "pd_joint_delta_pos")

Warp initialized:
   Version: 0.3.1
   CUDA device: NVIDIA GeForce GTX 1080 Ti
   Kernel cache: /home/yihe/.cache/warp/0.3.1


2022-08-05 16:24:56,265 - mani_skill2 - INFO - load sdfs from file


In [7]:
from moviepy.editor import ImageSequenceClip
def animate(imgs, fps=20):
    imgs = ImageSequenceClip(imgs, fps=fps)
    imgs.write_videofile('animate.mp4', fps=fps)

In [43]:
class ManiskillDemoDataset(Dataset):

    def __init__(self, h5_file) -> None:
        self.h5_demo = GDict.from_hdf5(h5_file)
        self.all_obs_act_pair = self.read_h5(self.h5_demo)
    
    def read_h5(self, h5_demo):
        all_pair = []
        count = 0
        for key in h5_demo.keys():
            traj = h5_demo[key]
            
            for i in range(len(traj['actions'])):
                all_pair.append((traj['obs']['xyz'][i].reshape(-1,), traj['actions'][i]))
            count += 1

            if count == 10:
                break
        return all_pair

    def __len__(self):
        return len(self.all_obs_act_pair)
    
    def __getitem__(self, index):
        return self.all_obs_act_pair[index]

In [44]:
dataset = ManiskillDemoDataset("/home/yihe/ManiSkill2022/examples/softbody_record_demo/demos/Hang-v0/final_200traj/trajectory.none.pd_joint_delta_pos.pcd.h5")

In [45]:
obs, act = dataset.__getitem__(0)

In [46]:
obs.shape

(6144,)

In [47]:
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)

In [48]:
for i_batch, sample_batched in enumerate(dataloader):
    obs, act = sample_batched
    print(obs.shape)
    print(act.shape)
    break

torch.Size([4, 6144])
torch.Size([4, 8])
